# Set Environment

In [100]:
# Import libraries
from datetime import timedelta
import os

import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas_gbq

from IPython.display import display, HTML, Image
%matplotlib inline

plt.style.use('ggplot')
plt.rcParams.update({'font.size': 20})

# Access data using Google BigQuery.
from google.colab import auth
from google.cloud import bigquery

In [101]:
# authenticate
auth.authenticate_user()

In [102]:
# Set up environment variables
project_id = 'majestic-bounty-432904-c4'
# if project_id == 'CHANGE-ME':
#   raise ValueError('You must change project_id to your GCP project.')
os.environ["GOOGLE_CLOUD_PROJECT"] = project_id

# Read data from BigQuery into pandas dataframes.
def run_query(query, project_id=project_id):
  return pandas_gbq.read_gbq(
      query,
      project_id=project_id,
      dialect='standard')

# set the dataset
# if you want to use the demo, change this to mimic_demo
dataset = 'mimiciv'

# Feature Selection

## Select stroke patients

In [103]:
select_stroke_query = f"""
SELECT *
FROM `physionet-data.mimiciv_hosp.diagnoses_icd`
WHERE icd_code LIKE 'I61%' OR icd_code LIKE 'I63%';
"""
stroke_df = run_query(select_stroke_query)
stroke_subject_ids = stroke_df['subject_id'].unique().tolist()
stroke_hadm_ids = stroke_df['hadm_id'].unique().tolist()

Downloading: 100%|██████████|


## Demographics

### admittime, dischtime, deathtime, insurance, martial status, race

In [104]:
select_demographics_query_in_admission = f"""
SELECT subject_id, hadm_id, admittime, dischtime, deathtime, insurance, marital_status, race
FROM `physionet-data.mimiciv_hosp.admissions`
WHERE subject_id IN ({', '.join(map(str, stroke_subject_ids))})
AND hadm_id IN ({', '.join(map(str, stroke_hadm_ids))});
"""
demographics_df = run_query(select_demographics_query_in_admission)
demographics_df

Downloading: 100%|██████████|


,subject_id,hadm_id,admittime,dischtime,deathtime,insurance,marital_status,race
0,16648184,28237094,2130-05-10 20:25:00,2130-05-14 15:17:00,2130-05-14 15:17:00,Medicare,SINGLE,WHITE
1,14417339,28899895,2166-01-06 11:11:00,2166-01-11 15:42:00,NaT,Medicare,None,UNKNOWN
2,17373919,27627470,2143-05-13 23:32:00,2143-05-19 17:55:00,NaT,Other,SINGLE,BLACK/AFRICAN AMERICAN
3,14466370,28844727,2123-09-22 02:41:00,2123-09-27 12:07:00,NaT,Medicare,MARRIED,WHITE
4,18410974,29299221,2156-01-26 05:16:00,2156-01-30 15:02:00,NaT,Medicaid,SINGLE,BLACK/CARIBBEAN ISLAND
...,...,...,...,...,...,...,...,...
3961,16964010,23920941,2158-01-13 16:23:00,2158-01-18 09:33:00,2158-01-18 09:33:00,Other,MARRIED,ASIAN - CHINESE
3962,17295261,29266081,2136-07-29 18:29:00,2136-08-03 17:45:00,NaT,Medicare,WIDOWED,HISPANIC/LATINO - DOMINICAN
3963,18389709,27334428,2171-11-11 17:05:00,2171-11-17 17:52:00,NaT,Other,SINGLE,PATIENT DECLINED TO ANSWER
3964,18965721,27282483,2140-11-29 22:51:00,2141-01-11 17:59:00,NaT,Other,SINGLE,WHITE


### gender, age

In [105]:

select_gender_and_age_query = f"""
SELECT subject_id,gender, anchor_age
FROM `physionet-data.mimiciv_hosp.patients`
WHERE subject_id IN ({', '.join(map(str, stroke_subject_ids))});
"""
gender_and_df = run_query(select_gender_and_age_query)
demographics_df = demographics_df.merge(gender_and_df, on=['subject_id'], how='inner')
demographics_df

Downloading: 100%|██████████|


,subject_id,hadm_id,admittime,dischtime,deathtime,insurance,marital_status,race,gender,anchor_age
0,16648184,28237094,2130-05-10 20:25:00,2130-05-14 15:17:00,2130-05-14 15:17:00,Medicare,SINGLE,WHITE,M,74
1,14417339,28899895,2166-01-06 11:11:00,2166-01-11 15:42:00,NaT,Medicare,None,UNKNOWN,M,87
2,17373919,27627470,2143-05-13 23:32:00,2143-05-19 17:55:00,NaT,Other,SINGLE,BLACK/AFRICAN AMERICAN,M,40
3,14466370,28844727,2123-09-22 02:41:00,2123-09-27 12:07:00,NaT,Medicare,MARRIED,WHITE,M,79
4,18410974,29299221,2156-01-26 05:16:00,2156-01-30 15:02:00,NaT,Medicaid,SINGLE,BLACK/CARIBBEAN ISLAND,M,52
...,...,...,...,...,...,...,...,...,...,...
3961,16964010,23920941,2158-01-13 16:23:00,2158-01-18 09:33:00,2158-01-18 09:33:00,Other,MARRIED,ASIAN - CHINESE,F,50
3962,17295261,29266081,2136-07-29 18:29:00,2136-08-03 17:45:00,NaT,Medicare,WIDOWED,HISPANIC/LATINO - DOMINICAN,F,76
3963,18389709,27334428,2171-11-11 17:05:00,2171-11-17 17:52:00,NaT,Other,SINGLE,PATIENT DECLINED TO ANSWER,M,48
3964,18965721,27282483,2140-11-29 22:51:00,2141-01-11 17:59:00,NaT,Other,SINGLE,WHITE,F,62


### weight

In [106]:
# 226512 Admission weight
select_weight_query = f"""SELECT subject_id, hadm_id, AVG(valuenum) as weight
FROM `physionet-data.mimiciv_icu.chartevents`
WHERE itemid = 226512 AND hadm_id IN ({', '.join(map(str, stroke_hadm_ids))})
GROUP BY subject_id, hadm_id;
"""
weight_df = run_query(select_weight_query)
weight_df

Downloading: 100%|██████████|


,subject_id,hadm_id,weight
0,16617024,27449764,63.0
1,19604256,24330860,68.4
2,17032851,26219826,86.0
3,19703655,26745117,65.0
4,11949271,23754970,98.0
...,...,...,...
2412,12915995,22467260,138.8
2413,15344490,23968904,54.0
2414,10604870,26753935,95.0
2415,18915457,28642620,88.1


In [107]:
demographics_df = demographics_df.merge(weight_df, on=['subject_id','hadm_id'], how='left')
demographics_df

,subject_id,hadm_id,admittime,dischtime,deathtime,insurance,marital_status,race,gender,anchor_age,weight
0,16648184,28237094,2130-05-10 20:25:00,2130-05-14 15:17:00,2130-05-14 15:17:00,Medicare,SINGLE,WHITE,M,74,131.0
1,14417339,28899895,2166-01-06 11:11:00,2166-01-11 15:42:00,NaT,Medicare,None,UNKNOWN,M,87,81.0
2,17373919,27627470,2143-05-13 23:32:00,2143-05-19 17:55:00,NaT,Other,SINGLE,BLACK/AFRICAN AMERICAN,M,40,95.4
3,14466370,28844727,2123-09-22 02:41:00,2123-09-27 12:07:00,NaT,Medicare,MARRIED,WHITE,M,79,NaN
4,18410974,29299221,2156-01-26 05:16:00,2156-01-30 15:02:00,NaT,Medicaid,SINGLE,BLACK/CARIBBEAN ISLAND,M,52,NaN
...,...,...,...,...,...,...,...,...,...,...,...
3961,16964010,23920941,2158-01-13 16:23:00,2158-01-18 09:33:00,2158-01-18 09:33:00,Other,MARRIED,ASIAN - CHINESE,F,50,75.6
3962,17295261,29266081,2136-07-29 18:29:00,2136-08-03 17:45:00,NaT,Medicare,WIDOWED,HISPANIC/LATINO - DOMINICAN,F,76,NaN
3963,18389709,27334428,2171-11-11 17:05:00,2171-11-17 17:52:00,NaT,Other,SINGLE,PATIENT DECLINED TO ANSWER,M,48,NaN
3964,18965721,27282483,2140-11-29 22:51:00,2141-01-11 17:59:00,NaT,Other,SINGLE,WHITE,F,62,92.0


## Vital signs

In [108]:
# 220045  heart rate
# 220210  Respiratory Rate
# 220050  Arterial Blood Pressure systolic
# 220051  Arterial Blood Pressure diastolic
# 220052  Arterial Blood Pressure mean
# 228232  PAR-Oxygen saturation

select_vital_signs_query = f"""
SELECT *
FROM `physionet-data.mimiciv_icu.d_items`
WHERE itemid IN (220045, 220210, 220050, 220051, 220052, 228232);
"""
vital_signs_query = run_query(select_vital_signs_query)
vital_signs_query


Downloading: 100%|██████████|


,itemid,label,abbreviation,linksto,category,unitname,param_type,lownormalvalue,highnormalvalue
0,220210,Respiratory Rate,RR,chartevents,Respiratory,insp/min,Numeric,NaN,NaN
1,228232,PAR-Oxygen saturation,PAR-Oxygen saturation,chartevents,Routine Vital Signs,None,Text,NaN,NaN
2,220045,Heart Rate,HR,chartevents,Routine Vital Signs,bpm,Numeric,NaN,NaN
3,220050,Arterial Blood Pressure systolic,ABPs,chartevents,Routine Vital Signs,mmHg,Numeric,90.0,140.0
4,220051,Arterial Blood Pressure diastolic,ABPd,chartevents,Routine Vital Signs,mmHg,Numeric,60.0,90.0
5,220052,Arterial Blood Pressure mean,ABPm,chartevents,Routine Vital Signs,mmHg,Numeric,NaN,NaN


### Heart Rate

In [109]:
select_hr_query = f"""SELECT subject_id, hadm_id, AVG(valuenum) as hr_value
FROM `physionet-data.mimiciv_icu.chartevents`
WHERE itemid = 220045 AND subject_id IN ({', '.join(map(str, stroke_subject_ids))}) AND hadm_id IN ({', '.join(map(str, stroke_hadm_ids))})
GROUP BY subject_id, hadm_id;
"""
hr_df = run_query(select_hr_query)
hr_df

Downloading: 100%|██████████|


,subject_id,hadm_id,hr_value
0,19672845,27439137,77.628319
1,18737268,26481793,72.777533
2,19753816,21137917,76.377049
3,19678538,28383469,104.263889
4,18537209,26564360,79.826568
...,...,...,...
2415,16456719,22321118,84.444444
2416,12956096,20924402,87.509434
2417,11075109,29389294,62.529412
2418,10356145,20548235,74.312500


In [110]:
demographics_vitial_signs_df = demographics_df.merge(hr_df, on=['subject_id','hadm_id'], how='left')
demographics_vitial_signs_df

,subject_id,hadm_id,admittime,dischtime,deathtime,insurance,marital_status,race,gender,anchor_age,weight,hr_value
0,16648184,28237094,2130-05-10 20:25:00,2130-05-14 15:17:00,2130-05-14 15:17:00,Medicare,SINGLE,WHITE,M,74,131.0,70.318519
1,14417339,28899895,2166-01-06 11:11:00,2166-01-11 15:42:00,NaT,Medicare,None,UNKNOWN,M,87,81.0,118.038168
2,17373919,27627470,2143-05-13 23:32:00,2143-05-19 17:55:00,NaT,Other,SINGLE,BLACK/AFRICAN AMERICAN,M,40,95.4,79.054054
3,14466370,28844727,2123-09-22 02:41:00,2123-09-27 12:07:00,NaT,Medicare,MARRIED,WHITE,M,79,NaN,NaN
4,18410974,29299221,2156-01-26 05:16:00,2156-01-30 15:02:00,NaT,Medicaid,SINGLE,BLACK/CARIBBEAN ISLAND,M,52,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
3961,16964010,23920941,2158-01-13 16:23:00,2158-01-18 09:33:00,2158-01-18 09:33:00,Other,MARRIED,ASIAN - CHINESE,F,50,75.6,84.944444
3962,17295261,29266081,2136-07-29 18:29:00,2136-08-03 17:45:00,NaT,Medicare,WIDOWED,HISPANIC/LATINO - DOMINICAN,F,76,NaN,NaN
3963,18389709,27334428,2171-11-11 17:05:00,2171-11-17 17:52:00,NaT,Other,SINGLE,PATIENT DECLINED TO ANSWER,M,48,NaN,NaN
3964,18965721,27282483,2140-11-29 22:51:00,2141-01-11 17:59:00,NaT,Other,SINGLE,WHITE,F,62,92.0,63.617347


### Respiratory Rate

In [111]:
select_rr_query = f"""SELECT subject_id, hadm_id, AVG(valuenum) as rr_value
FROM `physionet-data.mimiciv_icu.chartevents`
WHERE itemid = 220210 AND subject_id IN ({', '.join(map(str, stroke_subject_ids))}) AND hadm_id IN ({', '.join(map(str, stroke_hadm_ids))})
GROUP BY subject_id, hadm_id;
"""
rr_df = run_query(select_rr_query)
rr_df

Downloading: 100%|██████████|


,subject_id,hadm_id,rr_value
0,16331109,23234327,18.162791
1,16841093,21934758,18.807339
2,19724657,21086458,21.531646
3,11741391,24147037,20.258065
4,14376669,22578029,16.731707
...,...,...,...
2414,12972832,20571535,20.333333
2415,12026790,23977148,15.437500
2416,15838849,28975630,22.809524
2417,13982608,24642654,19.428571


In [112]:
demographics_vitial_signs_df = demographics_vitial_signs_df.merge(rr_df, on=['subject_id','hadm_id'], how='left')
demographics_vitial_signs_df

,subject_id,hadm_id,admittime,dischtime,deathtime,insurance,marital_status,race,gender,anchor_age,weight,hr_value,rr_value
0,16648184,28237094,2130-05-10 20:25:00,2130-05-14 15:17:00,2130-05-14 15:17:00,Medicare,SINGLE,WHITE,M,74,131.0,70.318519,15.644928
1,14417339,28899895,2166-01-06 11:11:00,2166-01-11 15:42:00,NaT,Medicare,None,UNKNOWN,M,87,81.0,118.038168,23.454545
2,17373919,27627470,2143-05-13 23:32:00,2143-05-19 17:55:00,NaT,Other,SINGLE,BLACK/AFRICAN AMERICAN,M,40,95.4,79.054054,13.436620
3,14466370,28844727,2123-09-22 02:41:00,2123-09-27 12:07:00,NaT,Medicare,MARRIED,WHITE,M,79,NaN,NaN,NaN
4,18410974,29299221,2156-01-26 05:16:00,2156-01-30 15:02:00,NaT,Medicaid,SINGLE,BLACK/CARIBBEAN ISLAND,M,52,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3961,16964010,23920941,2158-01-13 16:23:00,2158-01-18 09:33:00,2158-01-18 09:33:00,Other,MARRIED,ASIAN - CHINESE,F,50,75.6,84.944444,19.305556
3962,17295261,29266081,2136-07-29 18:29:00,2136-08-03 17:45:00,NaT,Medicare,WIDOWED,HISPANIC/LATINO - DOMINICAN,F,76,NaN,NaN,NaN
3963,18389709,27334428,2171-11-11 17:05:00,2171-11-17 17:52:00,NaT,Other,SINGLE,PATIENT DECLINED TO ANSWER,M,48,NaN,NaN,NaN
3964,18965721,27282483,2140-11-29 22:51:00,2141-01-11 17:59:00,NaT,Other,SINGLE,WHITE,F,62,92.0,63.617347,17.147959


### SPO2

In [113]:
select_SPO2_query = f"""SELECT subject_id, hadm_id, AVG(valuenum) as SPO2_value
FROM `physionet-data.mimiciv_icu.chartevents`
WHERE itemid = 228232 AND subject_id IN ({', '.join(map(str, stroke_subject_ids))}) AND hadm_id IN ({', '.join(map(str, stroke_hadm_ids))})
GROUP BY subject_id, hadm_id;
"""
SPO2_df = run_query(select_SPO2_query)
SPO2_df

Downloading: 100%|██████████|


,subject_id,hadm_id,SPO2_value
0,14470386,20124738,1.0
1,16293620,24841345,2.0
2,11121125,21867186,0.0
3,19404265,26466590,1.0
4,10565616,21618127,1.0
...,...,...,...
177,10975341,20428175,1.5
178,17123098,20351877,1.0
179,10722962,22516334,2.0
180,17176996,25317420,2.0


In [114]:
demographics_vitial_signs_df = demographics_vitial_signs_df.merge(SPO2_df, on=['subject_id','hadm_id'], how='left')
demographics_vitial_signs_df

,subject_id,hadm_id,admittime,dischtime,deathtime,insurance,marital_status,race,gender,anchor_age,weight,hr_value,rr_value,SPO2_value
0,16648184,28237094,2130-05-10 20:25:00,2130-05-14 15:17:00,2130-05-14 15:17:00,Medicare,SINGLE,WHITE,M,74,131.0,70.318519,15.644928,NaN
1,14417339,28899895,2166-01-06 11:11:00,2166-01-11 15:42:00,NaT,Medicare,None,UNKNOWN,M,87,81.0,118.038168,23.454545,NaN
2,17373919,27627470,2143-05-13 23:32:00,2143-05-19 17:55:00,NaT,Other,SINGLE,BLACK/AFRICAN AMERICAN,M,40,95.4,79.054054,13.436620,NaN
3,14466370,28844727,2123-09-22 02:41:00,2123-09-27 12:07:00,NaT,Medicare,MARRIED,WHITE,M,79,NaN,NaN,NaN,NaN
4,18410974,29299221,2156-01-26 05:16:00,2156-01-30 15:02:00,NaT,Medicaid,SINGLE,BLACK/CARIBBEAN ISLAND,M,52,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3961,16964010,23920941,2158-01-13 16:23:00,2158-01-18 09:33:00,2158-01-18 09:33:00,Other,MARRIED,ASIAN - CHINESE,F,50,75.6,84.944444,19.305556,NaN
3962,17295261,29266081,2136-07-29 18:29:00,2136-08-03 17:45:00,NaT,Medicare,WIDOWED,HISPANIC/LATINO - DOMINICAN,F,76,NaN,NaN,NaN,NaN
3963,18389709,27334428,2171-11-11 17:05:00,2171-11-17 17:52:00,NaT,Other,SINGLE,PATIENT DECLINED TO ANSWER,M,48,NaN,NaN,NaN,NaN
3964,18965721,27282483,2140-11-29 22:51:00,2141-01-11 17:59:00,NaT,Other,SINGLE,WHITE,F,62,92.0,63.617347,17.147959,NaN


### Arterial Blood Pressure systolic

In [115]:
select_BPS_query = f"""SELECT subject_id, hadm_id, AVG(valuenum) as BPS_value
FROM `physionet-data.mimiciv_icu.chartevents`
WHERE itemid = 220050 AND subject_id IN ({', '.join(map(str, stroke_subject_ids))}) AND hadm_id IN ({', '.join(map(str, stroke_hadm_ids))})
GROUP BY subject_id, hadm_id;
"""
BPS_df = run_query(select_BPS_query)
BPS_df

Downloading: 100%|██████████|


,subject_id,hadm_id,BPS_value
0,17011138,20902878,128.802548
1,18919724,20177875,139.052632
2,16417863,21732753,137.416667
3,19087971,26719664,119.129630
4,19366315,29723511,147.683761
...,...,...,...
751,14755254,28933877,117.529412
752,19208673,26514864,139.333333
753,16973388,25316143,112.111111
754,18749464,25359386,118.000000


In [116]:
demographics_vitial_signs_df = demographics_vitial_signs_df.merge(BPS_df, on=['subject_id','hadm_id'], how='left')
demographics_vitial_signs_df

,subject_id,hadm_id,admittime,dischtime,deathtime,insurance,marital_status,race,gender,anchor_age,weight,hr_value,rr_value,SPO2_value,BPS_value
0,16648184,28237094,2130-05-10 20:25:00,2130-05-14 15:17:00,2130-05-14 15:17:00,Medicare,SINGLE,WHITE,M,74,131.0,70.318519,15.644928,NaN,NaN
1,14417339,28899895,2166-01-06 11:11:00,2166-01-11 15:42:00,NaT,Medicare,None,UNKNOWN,M,87,81.0,118.038168,23.454545,NaN,128.6
2,17373919,27627470,2143-05-13 23:32:00,2143-05-19 17:55:00,NaT,Other,SINGLE,BLACK/AFRICAN AMERICAN,M,40,95.4,79.054054,13.436620,NaN,NaN
3,14466370,28844727,2123-09-22 02:41:00,2123-09-27 12:07:00,NaT,Medicare,MARRIED,WHITE,M,79,NaN,NaN,NaN,NaN,NaN
4,18410974,29299221,2156-01-26 05:16:00,2156-01-30 15:02:00,NaT,Medicaid,SINGLE,BLACK/CARIBBEAN ISLAND,M,52,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3961,16964010,23920941,2158-01-13 16:23:00,2158-01-18 09:33:00,2158-01-18 09:33:00,Other,MARRIED,ASIAN - CHINESE,F,50,75.6,84.944444,19.305556,NaN,NaN
3962,17295261,29266081,2136-07-29 18:29:00,2136-08-03 17:45:00,NaT,Medicare,WIDOWED,HISPANIC/LATINO - DOMINICAN,F,76,NaN,NaN,NaN,NaN,NaN
3963,18389709,27334428,2171-11-11 17:05:00,2171-11-17 17:52:00,NaT,Other,SINGLE,PATIENT DECLINED TO ANSWER,M,48,NaN,NaN,NaN,NaN,NaN
3964,18965721,27282483,2140-11-29 22:51:00,2141-01-11 17:59:00,NaT,Other,SINGLE,WHITE,F,62,92.0,63.617347,17.147959,NaN,NaN


### Arterial Blood Pressure diastolic

In [117]:
select_BPD_query = f"""SELECT subject_id, hadm_id, AVG(valuenum) as BPD_value
FROM `physionet-data.mimiciv_icu.chartevents`
WHERE itemid = 220051 AND subject_id IN ({', '.join(map(str, stroke_subject_ids))}) AND hadm_id IN ({', '.join(map(str, stroke_hadm_ids))})
GROUP BY subject_id, hadm_id;
"""
BPD_df = run_query(select_BPD_query)
BPD_df



Downloading: 100%|██████████|


,subject_id,hadm_id,BPD_value
0,17501806,25851598,50.461538
1,19928591,27383823,57.196970
2,16056209,25972241,71.000000
3,18334822,23794654,58.707143
4,16259359,28822852,55.256098
...,...,...,...
751,11029102,22904305,71.800000
752,14263190,29006576,56.229167
753,19310368,28920315,74.538462
754,10557131,22308370,59.527778


In [118]:
demographics_vitial_signs_df = demographics_vitial_signs_df.merge(BPD_df, on=['subject_id','hadm_id'], how='left')
demographics_vitial_signs_df

,subject_id,hadm_id,admittime,dischtime,deathtime,insurance,marital_status,race,gender,anchor_age,weight,hr_value,rr_value,SPO2_value,BPS_value,BPD_value
0,16648184,28237094,2130-05-10 20:25:00,2130-05-14 15:17:00,2130-05-14 15:17:00,Medicare,SINGLE,WHITE,M,74,131.0,70.318519,15.644928,NaN,NaN,NaN
1,14417339,28899895,2166-01-06 11:11:00,2166-01-11 15:42:00,NaT,Medicare,None,UNKNOWN,M,87,81.0,118.038168,23.454545,NaN,128.6,59.6
2,17373919,27627470,2143-05-13 23:32:00,2143-05-19 17:55:00,NaT,Other,SINGLE,BLACK/AFRICAN AMERICAN,M,40,95.4,79.054054,13.436620,NaN,NaN,NaN
3,14466370,28844727,2123-09-22 02:41:00,2123-09-27 12:07:00,NaT,Medicare,MARRIED,WHITE,M,79,NaN,NaN,NaN,NaN,NaN,NaN
4,18410974,29299221,2156-01-26 05:16:00,2156-01-30 15:02:00,NaT,Medicaid,SINGLE,BLACK/CARIBBEAN ISLAND,M,52,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3961,16964010,23920941,2158-01-13 16:23:00,2158-01-18 09:33:00,2158-01-18 09:33:00,Other,MARRIED,ASIAN - CHINESE,F,50,75.6,84.944444,19.305556,NaN,NaN,NaN
3962,17295261,29266081,2136-07-29 18:29:00,2136-08-03 17:45:00,NaT,Medicare,WIDOWED,HISPANIC/LATINO - DOMINICAN,F,76,NaN,NaN,NaN,NaN,NaN,NaN
3963,18389709,27334428,2171-11-11 17:05:00,2171-11-17 17:52:00,NaT,Other,SINGLE,PATIENT DECLINED TO ANSWER,M,48,NaN,NaN,NaN,NaN,NaN,NaN
3964,18965721,27282483,2140-11-29 22:51:00,2141-01-11 17:59:00,NaT,Other,SINGLE,WHITE,F,62,92.0,63.617347,17.147959,NaN,NaN,NaN


### Arterial Blood Pressure Mean

In [119]:
select_BPM_query = f"""SELECT subject_id, hadm_id, AVG(valuenum) as BPM_value
FROM `physionet-data.mimiciv_icu.chartevents`
WHERE itemid = 220052 AND subject_id IN ({', '.join(map(str, stroke_subject_ids))}) AND hadm_id IN ({', '.join(map(str, stroke_hadm_ids))})
GROUP BY subject_id, hadm_id;
"""
BPM_df = run_query(select_BPM_query)
BPM_df

Downloading: 100%|██████████|


,subject_id,hadm_id,BPM_value
0,17530357,24041886,81.410959
1,16462253,25773781,91.318275
2,16442940,23226100,84.862069
3,14230424,25782048,81.551724
4,16582052,27639797,67.562217
...,...,...,...
754,12464172,20245963,65.000000
755,11977608,20062363,82.477273
756,11572460,25285710,93.178571
757,10441831,24049550,99.000000


In [120]:
demographics_vitial_signs_df = demographics_vitial_signs_df.merge(BPM_df, on=['subject_id','hadm_id'], how='left')
demographics_vitial_signs_df

,subject_id,hadm_id,admittime,dischtime,deathtime,insurance,marital_status,race,gender,anchor_age,weight,hr_value,rr_value,SPO2_value,BPS_value,BPD_value,BPM_value
0,16648184,28237094,2130-05-10 20:25:00,2130-05-14 15:17:00,2130-05-14 15:17:00,Medicare,SINGLE,WHITE,M,74,131.0,70.318519,15.644928,NaN,NaN,NaN,NaN
1,14417339,28899895,2166-01-06 11:11:00,2166-01-11 15:42:00,NaT,Medicare,None,UNKNOWN,M,87,81.0,118.038168,23.454545,NaN,128.6,59.6,79.2
2,17373919,27627470,2143-05-13 23:32:00,2143-05-19 17:55:00,NaT,Other,SINGLE,BLACK/AFRICAN AMERICAN,M,40,95.4,79.054054,13.436620,NaN,NaN,NaN,NaN
3,14466370,28844727,2123-09-22 02:41:00,2123-09-27 12:07:00,NaT,Medicare,MARRIED,WHITE,M,79,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,18410974,29299221,2156-01-26 05:16:00,2156-01-30 15:02:00,NaT,Medicaid,SINGLE,BLACK/CARIBBEAN ISLAND,M,52,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3961,16964010,23920941,2158-01-13 16:23:00,2158-01-18 09:33:00,2158-01-18 09:33:00,Other,MARRIED,ASIAN - CHINESE,F,50,75.6,84.944444,19.305556,NaN,NaN,NaN,NaN
3962,17295261,29266081,2136-07-29 18:29:00,2136-08-03 17:45:00,NaT,Medicare,WIDOWED,HISPANIC/LATINO - DOMINICAN,F,76,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3963,18389709,27334428,2171-11-11 17:05:00,2171-11-17 17:52:00,NaT,Other,SINGLE,PATIENT DECLINED TO ANSWER,M,48,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3964,18965721,27282483,2140-11-29 22:51:00,2141-01-11 17:59:00,NaT,Other,SINGLE,WHITE,F,62,92.0,63.617347,17.147959,NaN,NaN,NaN,NaN


## Laboratory results

In [121]:
# Anion Gap 227073
# Chloride (serum)	220602

select_laboratory_results_query = f"""
SELECT *
FROM `physionet-data.mimiciv_icu.d_items`
WHERE LOWER(label) like '%chloride%'
"""
laboratory_results_query = run_query(select_laboratory_results_query)
laboratory_results_query

Downloading: 100%|██████████|


,itemid,label,abbreviation,linksto,category,unitname,param_type,lownormalvalue,highnormalvalue
0,220602,Chloride (serum),Chloride (serum),chartevents,Labs,None,Numeric,NaN,NaN
1,226536,Chloride (whole blood),Chloride (whole blood),chartevents,Labs,None,Numeric,NaN,NaN
2,228385,Chloride (serum) (soft),Chloride (serum) (soft),chartevents,Labs,None,Numeric with tag,NaN,NaN
3,228386,Chloride (whole blood) (soft),Chloride (whole blood) (soft),chartevents,Labs,None,Numeric with tag,NaN,NaN
4,225166,Potassium Chloride,Potassium Chloride - KCL,inputevents,Medications,mEq,Solution,NaN,NaN
5,227520,Methadone Hydrochloride,Methadone Hydrochloride,inputevents,Medications,mg,Solution,NaN,NaN
6,229618,Calcium Chloride,Calcium Chloride,inputevents,Medications,mg,Solution,NaN,NaN
7,220367,Chloride (ingr),Cl (ingr),ingredientevents,Ingredients - general (Not In Use),mg,Ingredient,NaN,NaN


### Anion Gap

In [122]:
select_AG_query = f"""
SELECT subject_id, hadm_id, AVG(AG_value) AS AG_value
FROM (
    SELECT ce.subject_id, ce.hadm_id, ce.valuenum AS AG_value
    FROM `physionet-data.mimiciv_icu.chartevents` AS ce
    JOIN `physionet-data.mimiciv_icu.icustays` AS icu
    ON ce.subject_id = icu.subject_id AND ce.hadm_id = icu.hadm_id
    WHERE ce.itemid = 227073
    AND TIMESTAMP_DIFF(ce.charttime, icu.intime, HOUR) <= 24
    AND ce.subject_id IN ({', '.join(map(str, stroke_subject_ids))})
    AND ce.hadm_id IN ({', '.join(map(str, stroke_hadm_ids))})
) AS filtered_data
GROUP BY subject_id, hadm_id;
"""
AG_df = run_query(select_AG_query)
AG_df

Downloading: 100%|██████████|


,subject_id,hadm_id,AG_value
0,10855190,29811371,14.333333
1,12560741,28949065,11.000000
2,13552058,26426627,17.433333
3,12022798,26030081,15.000000
4,17810802,22147008,12.000000
...,...,...,...
2277,17519740,20226467,19.000000
2278,16785545,22371541,19.000000
2279,19823136,20688923,10.000000
2280,19287751,25636491,19.000000


In [123]:
demo_vs_lab_df = demographics_vitial_signs_df.merge(AG_df, on=['subject_id','hadm_id'], how='left')
demo_vs_lab_df

,subject_id,hadm_id,admittime,dischtime,deathtime,insurance,marital_status,race,gender,anchor_age,weight,hr_value,rr_value,SPO2_value,BPS_value,BPD_value,BPM_value,AG_value
0,16648184,28237094,2130-05-10 20:25:00,2130-05-14 15:17:00,2130-05-14 15:17:00,Medicare,SINGLE,WHITE,M,74,131.0,70.318519,15.644928,NaN,NaN,NaN,NaN,17.00
1,14417339,28899895,2166-01-06 11:11:00,2166-01-11 15:42:00,NaT,Medicare,None,UNKNOWN,M,87,81.0,118.038168,23.454545,NaN,128.6,59.6,79.2,12.00
2,17373919,27627470,2143-05-13 23:32:00,2143-05-19 17:55:00,NaT,Other,SINGLE,BLACK/AFRICAN AMERICAN,M,40,95.4,79.054054,13.436620,NaN,NaN,NaN,NaN,16.50
3,14466370,28844727,2123-09-22 02:41:00,2123-09-27 12:07:00,NaT,Medicare,MARRIED,WHITE,M,79,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,18410974,29299221,2156-01-26 05:16:00,2156-01-30 15:02:00,NaT,Medicaid,SINGLE,BLACK/CARIBBEAN ISLAND,M,52,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3961,16964010,23920941,2158-01-13 16:23:00,2158-01-18 09:33:00,2158-01-18 09:33:00,Other,MARRIED,ASIAN - CHINESE,F,50,75.6,84.944444,19.305556,NaN,NaN,NaN,NaN,15.00
3962,17295261,29266081,2136-07-29 18:29:00,2136-08-03 17:45:00,NaT,Medicare,WIDOWED,HISPANIC/LATINO - DOMINICAN,F,76,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3963,18389709,27334428,2171-11-11 17:05:00,2171-11-17 17:52:00,NaT,Other,SINGLE,PATIENT DECLINED TO ANSWER,M,48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3964,18965721,27282483,2140-11-29 22:51:00,2141-01-11 17:59:00,NaT,Other,SINGLE,WHITE,F,62,92.0,63.617347,17.147959,NaN,NaN,NaN,NaN,14.75


### Chloride (serum)

In [124]:
select_chloride_query = f"""
SELECT subject_id, hadm_id, AVG(chloride_value) AS chloride_value
FROM (
    SELECT ce.subject_id, ce.hadm_id, ce.valuenum AS chloride_value
    FROM `physionet-data.mimiciv_icu.chartevents` AS ce
    JOIN `physionet-data.mimiciv_icu.icustays` AS icu
    ON ce.subject_id = icu.subject_id AND ce.hadm_id = icu.hadm_id
    WHERE ce.itemid = 220602
    AND TIMESTAMP_DIFF(ce.charttime, icu.intime, HOUR) <= 24
    AND ce.subject_id IN ({', '.join(map(str, stroke_subject_ids))})
    AND ce.hadm_id IN ({', '.join(map(str, stroke_hadm_ids))})
) AS filtered_data
GROUP BY subject_id, hadm_id;
"""
chloride_df = run_query(select_chloride_query)
chloride_df

Downloading: 100%|██████████|


,subject_id,hadm_id,chloride_value
0,16078400,27997577,94.5
1,16020425,26973298,99.0
2,13140596,23452264,101.0
3,17563347,24453981,108.5
4,15163147,27282226,106.0
...,...,...,...
2277,14944990,22228800,100.0
2278,12452689,23315901,107.0
2279,15556698,26628429,106.0
2280,19949052,28096518,104.0


In [125]:
demo_vs_lab_df = demo_vs_lab_df.merge(chloride_df, on=['subject_id','hadm_id'], how='left')
demo_vs_lab_df

,subject_id,hadm_id,admittime,dischtime,deathtime,insurance,marital_status,race,gender,anchor_age,weight,hr_value,rr_value,SPO2_value,BPS_value,BPD_value,BPM_value,AG_value,chloride_value
0,16648184,28237094,2130-05-10 20:25:00,2130-05-14 15:17:00,2130-05-14 15:17:00,Medicare,SINGLE,WHITE,M,74,131.0,70.318519,15.644928,NaN,NaN,NaN,NaN,17.00,98.00
1,14417339,28899895,2166-01-06 11:11:00,2166-01-11 15:42:00,NaT,Medicare,None,UNKNOWN,M,87,81.0,118.038168,23.454545,NaN,128.6,59.6,79.2,12.00,107.00
2,17373919,27627470,2143-05-13 23:32:00,2143-05-19 17:55:00,NaT,Other,SINGLE,BLACK/AFRICAN AMERICAN,M,40,95.4,79.054054,13.436620,NaN,NaN,NaN,NaN,16.50,100.25
3,14466370,28844727,2123-09-22 02:41:00,2123-09-27 12:07:00,NaT,Medicare,MARRIED,WHITE,M,79,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,18410974,29299221,2156-01-26 05:16:00,2156-01-30 15:02:00,NaT,Medicaid,SINGLE,BLACK/CARIBBEAN ISLAND,M,52,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3961,16964010,23920941,2158-01-13 16:23:00,2158-01-18 09:33:00,2158-01-18 09:33:00,Other,MARRIED,ASIAN - CHINESE,F,50,75.6,84.944444,19.305556,NaN,NaN,NaN,NaN,15.00,86.50
3962,17295261,29266081,2136-07-29 18:29:00,2136-08-03 17:45:00,NaT,Medicare,WIDOWED,HISPANIC/LATINO - DOMINICAN,F,76,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3963,18389709,27334428,2171-11-11 17:05:00,2171-11-17 17:52:00,NaT,Other,SINGLE,PATIENT DECLINED TO ANSWER,M,48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3964,18965721,27282483,2140-11-29 22:51:00,2141-01-11 17:59:00,NaT,Other,SINGLE,WHITE,F,62,92.0,63.617347,17.147959,NaN,NaN,NaN,NaN,14.75,93.75


### Bicarbonate

In [126]:
# Red Blood Cells counts 51279
# White Blood Cells 51301
# Bicarbonate 50882

select_bicarbonate_query = f"""SELECT subject_id, hadm_id, AVG(valuenum) AS bicarbonate_value
FROM `physionet-data.mimiciv_hosp.labevents`
WHERE itemid = 50882 AND subject_id IN ({', '.join(map(str, stroke_subject_ids))}) AND hadm_id IN ({', '.join(map(str, stroke_hadm_ids))})
group by subject_id, hadm_id;
"""
bicarbonate_df = run_query(select_bicarbonate_query)
bicarbonate_df


Downloading: 100%|██████████|


,subject_id,hadm_id,bicarbonate_value
0,10122772,23565074,23.954545
1,10215159,24039782,27.064516
2,10236108,21437036,25.736842
3,10300586,24090224,30.134615
4,10327156,29595456,24.857143
...,...,...,...
3823,10867818,21716308,23.000000
3824,14829995,29387162,25.000000
3825,19625064,22781169,24.000000
3826,14233727,24401115,25.000000


In [127]:
demo_vs_lab_df = demo_vs_lab_df.merge(bicarbonate_df, on=['subject_id','hadm_id'], how='left')
demo_vs_lab_df

,subject_id,hadm_id,admittime,dischtime,deathtime,insurance,marital_status,race,gender,anchor_age,weight,hr_value,rr_value,SPO2_value,BPS_value,BPD_value,BPM_value,AG_value,chloride_value,bicarbonate_value
0,16648184,28237094,2130-05-10 20:25:00,2130-05-14 15:17:00,2130-05-14 15:17:00,Medicare,SINGLE,WHITE,M,74,131.0,70.318519,15.644928,NaN,NaN,NaN,NaN,17.00,98.00,25.428571
1,14417339,28899895,2166-01-06 11:11:00,2166-01-11 15:42:00,NaT,Medicare,None,UNKNOWN,M,87,81.0,118.038168,23.454545,NaN,128.6,59.6,79.2,12.00,107.00,25.333333
2,17373919,27627470,2143-05-13 23:32:00,2143-05-19 17:55:00,NaT,Other,SINGLE,BLACK/AFRICAN AMERICAN,M,40,95.4,79.054054,13.436620,NaN,NaN,NaN,NaN,16.50,100.25,20.200000
3,14466370,28844727,2123-09-22 02:41:00,2123-09-27 12:07:00,NaT,Medicare,MARRIED,WHITE,M,79,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.600000
4,18410974,29299221,2156-01-26 05:16:00,2156-01-30 15:02:00,NaT,Medicaid,SINGLE,BLACK/CARIBBEAN ISLAND,M,52,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.166667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3961,16964010,23920941,2158-01-13 16:23:00,2158-01-18 09:33:00,2158-01-18 09:33:00,Other,MARRIED,ASIAN - CHINESE,F,50,75.6,84.944444,19.305556,NaN,NaN,NaN,NaN,15.00,86.50,39.400000
3962,17295261,29266081,2136-07-29 18:29:00,2136-08-03 17:45:00,NaT,Medicare,WIDOWED,HISPANIC/LATINO - DOMINICAN,F,76,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.000000
3963,18389709,27334428,2171-11-11 17:05:00,2171-11-17 17:52:00,NaT,Other,SINGLE,PATIENT DECLINED TO ANSWER,M,48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.500000
3964,18965721,27282483,2140-11-29 22:51:00,2141-01-11 17:59:00,NaT,Other,SINGLE,WHITE,F,62,92.0,63.617347,17.147959,NaN,NaN,NaN,NaN,14.75,93.75,29.041667


### Red Blood Cells counts

In [128]:

select_RBC_query = f"""SELECT subject_id, hadm_id, AVG(valuenum) AS RBC_value
FROM `physionet-data.mimiciv_hosp.labevents`
WHERE itemid = 51279 AND subject_id IN ({', '.join(map(str, stroke_subject_ids))}) AND hadm_id IN ({', '.join(map(str, stroke_hadm_ids))})
group by subject_id, hadm_id;
"""
RBC_df = run_query(select_RBC_query)
RBC_df

Downloading: 100%|██████████|


,subject_id,hadm_id,RBC_value
0,10057482,25416257,3.169216
1,10167784,26706672,3.062593
2,10223705,23264106,3.872857
3,10410774,21942478,3.184000
4,10667727,24639827,3.210400
...,...,...,...
3817,16598955,24875339,4.790000
3818,12326925,23182129,4.550000
3819,19726617,22728930,3.860000
3820,12038552,26561892,5.400000


In [129]:
demo_vs_lab_df = demo_vs_lab_df.merge(RBC_df, on=['subject_id','hadm_id'], how='left')
demo_vs_lab_df

,subject_id,hadm_id,admittime,dischtime,deathtime,insurance,marital_status,race,gender,anchor_age,...,hr_value,rr_value,SPO2_value,BPS_value,BPD_value,BPM_value,AG_value,chloride_value,bicarbonate_value,RBC_value
0,16648184,28237094,2130-05-10 20:25:00,2130-05-14 15:17:00,2130-05-14 15:17:00,Medicare,SINGLE,WHITE,M,74,...,70.318519,15.644928,NaN,NaN,NaN,NaN,17.00,98.00,25.428571,3.370000
1,14417339,28899895,2166-01-06 11:11:00,2166-01-11 15:42:00,NaT,Medicare,None,UNKNOWN,M,87,...,118.038168,23.454545,NaN,128.6,59.6,79.2,12.00,107.00,25.333333,2.926250
2,17373919,27627470,2143-05-13 23:32:00,2143-05-19 17:55:00,NaT,Other,SINGLE,BLACK/AFRICAN AMERICAN,M,40,...,79.054054,13.436620,NaN,NaN,NaN,NaN,16.50,100.25,20.200000,2.492500
3,14466370,28844727,2123-09-22 02:41:00,2123-09-27 12:07:00,NaT,Medicare,MARRIED,WHITE,M,79,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.600000,4.416000
4,18410974,29299221,2156-01-26 05:16:00,2156-01-30 15:02:00,NaT,Medicaid,SINGLE,BLACK/CARIBBEAN ISLAND,M,52,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.166667,2.872000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3961,16964010,23920941,2158-01-13 16:23:00,2158-01-18 09:33:00,2158-01-18 09:33:00,Other,MARRIED,ASIAN - CHINESE,F,50,...,84.944444,19.305556,NaN,NaN,NaN,NaN,15.00,86.50,39.400000,4.035714
3962,17295261,29266081,2136-07-29 18:29:00,2136-08-03 17:45:00,NaT,Medicare,WIDOWED,HISPANIC/LATINO - DOMINICAN,F,76,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.000000,4.370000
3963,18389709,27334428,2171-11-11 17:05:00,2171-11-17 17:52:00,NaT,Other,SINGLE,PATIENT DECLINED TO ANSWER,M,48,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.500000,5.026667
3964,18965721,27282483,2140-11-29 22:51:00,2141-01-11 17:59:00,NaT,Other,SINGLE,WHITE,F,62,...,63.617347,17.147959,NaN,NaN,NaN,NaN,14.75,93.75,29.041667,3.148723


### White Blood Cells counts

In [130]:
select_WBC_query = f"""SELECT subject_id, hadm_id, AVG(valuenum) AS WBC_value
FROM `physionet-data.mimiciv_hosp.labevents`
WHERE itemid = 51279 AND subject_id IN ({', '.join(map(str, stroke_subject_ids))}) AND hadm_id IN ({', '.join(map(str, stroke_hadm_ids))})
group by subject_id, hadm_id;
"""
WBC_df = run_query(select_WBC_query)
WBC_df

Downloading: 100%|██████████|


,subject_id,hadm_id,WBC_value
0,10030753,27165162,2.609333
1,10057482,25416257,3.169216
2,10662387,27559370,3.660000
3,10667727,24639827,3.210400
4,10698984,28947835,3.800000
...,...,...,...
3817,12688610,20759422,3.510000
3818,19755076,23197885,4.580000
3819,11085838,26061438,3.230000
3820,16682729,28332408,4.920000


In [131]:
demo_vs_lab_df = demo_vs_lab_df.merge(WBC_df, on=['subject_id','hadm_id'], how='left')
demo_vs_lab_df

,subject_id,hadm_id,admittime,dischtime,deathtime,insurance,marital_status,race,gender,anchor_age,...,rr_value,SPO2_value,BPS_value,BPD_value,BPM_value,AG_value,chloride_value,bicarbonate_value,RBC_value,WBC_value
0,16648184,28237094,2130-05-10 20:25:00,2130-05-14 15:17:00,2130-05-14 15:17:00,Medicare,SINGLE,WHITE,M,74,...,15.644928,NaN,NaN,NaN,NaN,17.00,98.00,25.428571,3.370000,3.370000
1,14417339,28899895,2166-01-06 11:11:00,2166-01-11 15:42:00,NaT,Medicare,None,UNKNOWN,M,87,...,23.454545,NaN,128.6,59.6,79.2,12.00,107.00,25.333333,2.926250,2.926250
2,17373919,27627470,2143-05-13 23:32:00,2143-05-19 17:55:00,NaT,Other,SINGLE,BLACK/AFRICAN AMERICAN,M,40,...,13.436620,NaN,NaN,NaN,NaN,16.50,100.25,20.200000,2.492500,2.492500
3,14466370,28844727,2123-09-22 02:41:00,2123-09-27 12:07:00,NaT,Medicare,MARRIED,WHITE,M,79,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.600000,4.416000,4.416000
4,18410974,29299221,2156-01-26 05:16:00,2156-01-30 15:02:00,NaT,Medicaid,SINGLE,BLACK/CARIBBEAN ISLAND,M,52,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.166667,2.872000,2.872000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3961,16964010,23920941,2158-01-13 16:23:00,2158-01-18 09:33:00,2158-01-18 09:33:00,Other,MARRIED,ASIAN - CHINESE,F,50,...,19.305556,NaN,NaN,NaN,NaN,15.00,86.50,39.400000,4.035714,4.035714
3962,17295261,29266081,2136-07-29 18:29:00,2136-08-03 17:45:00,NaT,Medicare,WIDOWED,HISPANIC/LATINO - DOMINICAN,F,76,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.000000,4.370000,4.370000
3963,18389709,27334428,2171-11-11 17:05:00,2171-11-17 17:52:00,NaT,Other,SINGLE,PATIENT DECLINED TO ANSWER,M,48,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.500000,5.026667,5.026667
3964,18965721,27282483,2140-11-29 22:51:00,2141-01-11 17:59:00,NaT,Other,SINGLE,WHITE,F,62,...,17.147959,NaN,NaN,NaN,NaN,14.75,93.75,29.041667,3.148723,3.148723


## assessment scale scores

In [132]:
# 220739 GCS - Eye Opening
# 223900 GCS - Verbal Response
# 223901 GCS - Motor Response
select_assessment_scores_query = f"""
SELECT *
FROM `physionet-data.mimiciv_icu.d_items`
WHERE LOWER(label) like '%gcs%'
"""
assessment_scores_query = run_query(select_assessment_scores_query)
assessment_scores_query

Downloading: 100%|██████████|


,itemid,label,abbreviation,linksto,category,unitname,param_type,lownormalvalue,highnormalvalue
0,220739,GCS - Eye Opening,Eye Opening,chartevents,Neurological,None,Text,NaN,NaN
1,223900,GCS - Verbal Response,Verbal Response,chartevents,Neurological,None,Text,NaN,NaN
2,223901,GCS - Motor Response,Motor Response,chartevents,Neurological,None,Text,NaN,NaN
3,226756,GCSEyeApacheIIValue,GCSEyeApacheIIValue,chartevents,Scores - APACHE II,None,Text,NaN,NaN
4,226757,GCSMotorApacheIIValue,GCSMotorApacheIIValue,chartevents,Scores - APACHE II,None,Text,NaN,NaN
5,226758,GCSVerbalApacheIIValue,GCSVerbalApacheIIValue,chartevents,Scores - APACHE II,None,Text,NaN,NaN
6,228112,GCSVerbalApacheIIValue (intubated),GCSVerbalApacheIIValue (intubated),chartevents,Scores - APACHE II,None,Text,NaN,NaN
7,226755,GcsApacheIIScore,GcsApacheIIScore,chartevents,Scores - APACHE II,None,Numeric,NaN,NaN
8,227011,GCSEye_ApacheIV,GCSEye_ApacheIV,chartevents,Scores - APACHE IV (2),None,Text,NaN,NaN
9,227012,GCSMotor_ApacheIV,GCSMotor_ApacheIV,chartevents,Scores - APACHE IV (2),None,Text,NaN,NaN


In [133]:
select_GCS_eye_query = f"""
SELECT subject_id, hadm_id, AVG(GCS_eye_opening_score) AS GCS_eye_opening_score
FROM (
    SELECT ce.subject_id, ce.hadm_id, ce.valuenum AS GCS_eye_opening_score
    FROM `physionet-data.mimiciv_icu.chartevents` AS ce
    JOIN `physionet-data.mimiciv_icu.icustays` AS icu
    ON ce.subject_id = icu.subject_id AND ce.hadm_id = icu.hadm_id
    WHERE ce.itemid = 220739
    AND TIMESTAMP_DIFF(ce.charttime, icu.intime, HOUR) <= 24
    AND ce.subject_id IN ({', '.join(map(str, stroke_subject_ids))})
    AND ce.hadm_id IN ({', '.join(map(str, stroke_hadm_ids))})
) AS filtered_data
GROUP BY subject_id, hadm_id;
"""
GCS_eye_df = run_query(select_GCS_eye_query)
GCS_eye_df

Downloading: 100%|██████████|


,subject_id,hadm_id,GCS_eye_opening_score
0,18837678,25688441,1.941176
1,19599798,25306345,3.486111
2,10689776,29681373,4.000000
3,11128068,20262180,3.166667
4,15641119,21650140,3.920000
...,...,...,...
2411,15152799,21134374,4.000000
2412,10488542,24635169,3.250000
2413,17494417,24730278,3.000000
2414,18310386,29044661,3.500000


In [134]:
select_GCS_verbal_query = f"""
SELECT subject_id, hadm_id, AVG(GCS_verbal_score) AS GCS_verbal_score
FROM (
    SELECT ce.subject_id, ce.hadm_id, ce.valuenum AS GCS_verbal_score
    FROM `physionet-data.mimiciv_icu.chartevents` AS ce
    JOIN `physionet-data.mimiciv_icu.icustays` AS icu
    ON ce.subject_id = icu.subject_id AND ce.hadm_id = icu.hadm_id
    WHERE ce.itemid = 223900
    AND TIMESTAMP_DIFF(ce.charttime, icu.intime, HOUR) <= 24
    AND ce.subject_id IN ({', '.join(map(str, stroke_subject_ids))})
    AND ce.hadm_id IN ({', '.join(map(str, stroke_hadm_ids))})
) AS filtered_data
GROUP BY subject_id, hadm_id;
"""
GCS_verbal_df = run_query(select_GCS_verbal_query)
GCS_verbal_df

Downloading: 100%|██████████|


,subject_id,hadm_id,GCS_verbal_score
0,15972728,25902102,2.084906
1,16238160,20108746,5.000000
2,19549572,26728667,4.500000
3,18418109,24532060,1.000000
4,17173309,22443690,1.555024
...,...,...,...
2411,15444417,24062517,1.000000
2412,18834457,24117340,5.000000
2413,18328540,22802979,5.000000
2414,11669818,29029759,1.000000


In [135]:
select_GCS_Motor_query = f"""
SELECT subject_id, hadm_id, AVG(GCS_Motor_score) AS GCS_Motor_score
FROM (
    SELECT ce.subject_id, ce.hadm_id, ce.valuenum AS GCS_Motor_score
    FROM `physionet-data.mimiciv_icu.chartevents` AS ce
    JOIN `physionet-data.mimiciv_icu.icustays` AS icu
    ON ce.subject_id = icu.subject_id AND ce.hadm_id = icu.hadm_id
    WHERE ce.itemid = 223901
    AND TIMESTAMP_DIFF(ce.charttime, icu.intime, HOUR) <= 24
    AND ce.subject_id IN ({', '.join(map(str, stroke_subject_ids))})
    AND ce.hadm_id IN ({', '.join(map(str, stroke_hadm_ids))})
) AS filtered_data
GROUP BY subject_id, hadm_id;
"""
GCS_Motor_df = run_query(select_GCS_Motor_query)
GCS_Motor_df

Downloading: 100%|██████████|


,subject_id,hadm_id,GCS_Motor_score
0,11707036,28722502,2.142857
1,18174687,22350899,5.228155
2,16348359,26032183,5.459016
3,16802438,22838118,5.675676
4,15972728,25902102,5.467290
...,...,...,...
2411,17735901,26597331,4.000000
2412,10441831,24049550,5.333333
2413,11953908,21366986,2.500000
2414,12153629,21181787,6.000000


In [136]:
GCS_score = GCS_eye_df.merge(GCS_verbal_df, on=['subject_id', 'hadm_id']).merge(GCS_Motor_df, on=['subject_id', 'hadm_id'])
GCS_score['GCS_score'] = GCS_score[['GCS_eye_opening_score', 'GCS_verbal_score', 'GCS_Motor_score']].sum(axis=1)
GCS_score = GCS_score.drop(columns=['GCS_eye_opening_score', 'GCS_verbal_score', 'GCS_Motor_score'])
GCS_score

,subject_id,hadm_id,GCS_score
0,18837678,25688441,9.764706
1,19599798,25306345,12.919841
2,10689776,29681373,15.000000
3,11128068,20262180,8.250000
4,15641119,21650140,14.920000
...,...,...,...
2411,15152799,21134374,15.000000
2412,10488542,24635169,11.500000
2413,17494417,24730278,10.000000
2414,18310386,29044661,13.500000


## Merged all features

In [137]:
merged_df = demo_vs_lab_df.merge(GCS_score, on=['subject_id','hadm_id'], how='left')
merged_df

,subject_id,hadm_id,admittime,dischtime,deathtime,insurance,marital_status,race,gender,anchor_age,...,SPO2_value,BPS_value,BPD_value,BPM_value,AG_value,chloride_value,bicarbonate_value,RBC_value,WBC_value,GCS_score
0,16648184,28237094,2130-05-10 20:25:00,2130-05-14 15:17:00,2130-05-14 15:17:00,Medicare,SINGLE,WHITE,M,74,...,NaN,NaN,NaN,NaN,17.00,98.00,25.428571,3.370000,3.370000,3.375000
1,14417339,28899895,2166-01-06 11:11:00,2166-01-11 15:42:00,NaT,Medicare,None,UNKNOWN,M,87,...,NaN,128.6,59.6,79.2,12.00,107.00,25.333333,2.926250,2.926250,6.833333
2,17373919,27627470,2143-05-13 23:32:00,2143-05-19 17:55:00,NaT,Other,SINGLE,BLACK/AFRICAN AMERICAN,M,40,...,NaN,NaN,NaN,NaN,16.50,100.25,20.200000,2.492500,2.492500,14.400000
3,14466370,28844727,2123-09-22 02:41:00,2123-09-27 12:07:00,NaT,Medicare,MARRIED,WHITE,M,79,...,NaN,NaN,NaN,NaN,NaN,NaN,26.600000,4.416000,4.416000,NaN
4,18410974,29299221,2156-01-26 05:16:00,2156-01-30 15:02:00,NaT,Medicaid,SINGLE,BLACK/CARIBBEAN ISLAND,M,52,...,NaN,NaN,NaN,NaN,NaN,NaN,27.166667,2.872000,2.872000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3961,16964010,23920941,2158-01-13 16:23:00,2158-01-18 09:33:00,2158-01-18 09:33:00,Other,MARRIED,ASIAN - CHINESE,F,50,...,NaN,NaN,NaN,NaN,15.00,86.50,39.400000,4.035714,4.035714,3.000000
3962,17295261,29266081,2136-07-29 18:29:00,2136-08-03 17:45:00,NaT,Medicare,WIDOWED,HISPANIC/LATINO - DOMINICAN,F,76,...,NaN,NaN,NaN,NaN,NaN,NaN,23.000000,4.370000,4.370000,NaN
3963,18389709,27334428,2171-11-11 17:05:00,2171-11-17 17:52:00,NaT,Other,SINGLE,PATIENT DECLINED TO ANSWER,M,48,...,NaN,NaN,NaN,NaN,NaN,NaN,26.500000,5.026667,5.026667,NaN
3964,18965721,27282483,2140-11-29 22:51:00,2141-01-11 17:59:00,NaT,Other,SINGLE,WHITE,F,62,...,NaN,NaN,NaN,NaN,14.75,93.75,29.041667,3.148723,3.148723,7.826087


# Pre-processing

In [138]:
na_columns = merged_df.isna().sum()
na_columns_with_na = na_columns[na_columns > 0]
print(na_columns_with_na)


deathtime            3455
marital_status        426
weight               1549
hr_value             1546
rr_value             1547
SPO2_value           3784
BPS_value            3210
BPD_value            3210
BPM_value            3207
AG_value             1684
chloride_value       1684
bicarbonate_value     138
RBC_value             144
WBC_value             144
GCS_score            1550
dtype: int64


## Convert Death time to 1 and 0

In [139]:
merged_df.loc[merged_df['deathtime'].notna(),'dod'] = int(1)
merged_df.loc[merged_df['deathtime'].isnull(),'dod'] = int(0)
merged_df['dod'] = merged_df['dod'].astype(int)
merged_df.drop(columns=['deathtime'], inplace=True)
merged_df.shape

(3966, 23)

## Handle missing values

In [140]:
def create_check_test_columns(df, columns):

    for col in columns:
        # Create a new test column, True indicates the value is present, False indicates missing
        df[f'test_{col}'] = df[col].notna()
    return df

def fill_na_with_zero(df, columns):
  for col in columns:
        # Fill missing values with 0
        df[col] = df[col].fillna(0)
  return df

# List of columns to be processed
columns_to_handle = ['AG_value', 'chloride_value', 'bicarbonate_value', 'RBC_value', 'WBC_value',
                     'marital_status', 'weight', 'hr_value', 'rr_value', 'SPO2_value',
                     'BPS_value', 'BPD_value', 'BPM_value','GCS_score']

# Call the function to handle missing values
merged_df = create_check_test_columns(merged_df, columns_to_handle)
merged_df

,subject_id,hadm_id,admittime,dischtime,insurance,marital_status,race,gender,anchor_age,weight,...,test_WBC_value,test_marital_status,test_weight,test_hr_value,test_rr_value,test_SPO2_value,test_BPS_value,test_BPD_value,test_BPM_value,test_GCS_score
0,16648184,28237094,2130-05-10 20:25:00,2130-05-14 15:17:00,Medicare,SINGLE,WHITE,M,74,131.0,...,True,True,True,True,True,False,False,False,False,True
1,14417339,28899895,2166-01-06 11:11:00,2166-01-11 15:42:00,Medicare,None,UNKNOWN,M,87,81.0,...,True,False,True,True,True,False,True,True,True,True
2,17373919,27627470,2143-05-13 23:32:00,2143-05-19 17:55:00,Other,SINGLE,BLACK/AFRICAN AMERICAN,M,40,95.4,...,True,True,True,True,True,False,False,False,False,True
3,14466370,28844727,2123-09-22 02:41:00,2123-09-27 12:07:00,Medicare,MARRIED,WHITE,M,79,NaN,...,True,True,False,False,False,False,False,False,False,False
4,18410974,29299221,2156-01-26 05:16:00,2156-01-30 15:02:00,Medicaid,SINGLE,BLACK/CARIBBEAN ISLAND,M,52,NaN,...,True,True,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3961,16964010,23920941,2158-01-13 16:23:00,2158-01-18 09:33:00,Other,MARRIED,ASIAN - CHINESE,F,50,75.6,...,True,True,True,True,True,False,False,False,False,True
3962,17295261,29266081,2136-07-29 18:29:00,2136-08-03 17:45:00,Medicare,WIDOWED,HISPANIC/LATINO - DOMINICAN,F,76,NaN,...,True,True,False,False,False,False,False,False,False,False
3963,18389709,27334428,2171-11-11 17:05:00,2171-11-17 17:52:00,Other,SINGLE,PATIENT DECLINED TO ANSWER,M,48,NaN,...,True,True,False,False,False,False,False,False,False,False
3964,18965721,27282483,2140-11-29 22:51:00,2141-01-11 17:59:00,Other,SINGLE,WHITE,F,62,92.0,...,True,True,True,True,True,False,False,False,False,True


In [141]:
merged_df = fill_na_with_zero(merged_df, columns_to_handle)
merged_df

,subject_id,hadm_id,admittime,dischtime,insurance,marital_status,race,gender,anchor_age,weight,...,test_WBC_value,test_marital_status,test_weight,test_hr_value,test_rr_value,test_SPO2_value,test_BPS_value,test_BPD_value,test_BPM_value,test_GCS_score
0,16648184,28237094,2130-05-10 20:25:00,2130-05-14 15:17:00,Medicare,SINGLE,WHITE,M,74,131.0,...,True,True,True,True,True,False,False,False,False,True
1,14417339,28899895,2166-01-06 11:11:00,2166-01-11 15:42:00,Medicare,0,UNKNOWN,M,87,81.0,...,True,False,True,True,True,False,True,True,True,True
2,17373919,27627470,2143-05-13 23:32:00,2143-05-19 17:55:00,Other,SINGLE,BLACK/AFRICAN AMERICAN,M,40,95.4,...,True,True,True,True,True,False,False,False,False,True
3,14466370,28844727,2123-09-22 02:41:00,2123-09-27 12:07:00,Medicare,MARRIED,WHITE,M,79,0.0,...,True,True,False,False,False,False,False,False,False,False
4,18410974,29299221,2156-01-26 05:16:00,2156-01-30 15:02:00,Medicaid,SINGLE,BLACK/CARIBBEAN ISLAND,M,52,0.0,...,True,True,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3961,16964010,23920941,2158-01-13 16:23:00,2158-01-18 09:33:00,Other,MARRIED,ASIAN - CHINESE,F,50,75.6,...,True,True,True,True,True,False,False,False,False,True
3962,17295261,29266081,2136-07-29 18:29:00,2136-08-03 17:45:00,Medicare,WIDOWED,HISPANIC/LATINO - DOMINICAN,F,76,0.0,...,True,True,False,False,False,False,False,False,False,False
3963,18389709,27334428,2171-11-11 17:05:00,2171-11-17 17:52:00,Other,SINGLE,PATIENT DECLINED TO ANSWER,M,48,0.0,...,True,True,False,False,False,False,False,False,False,False
3964,18965721,27282483,2140-11-29 22:51:00,2141-01-11 17:59:00,Other,SINGLE,WHITE,F,62,92.0,...,True,True,True,True,True,False,False,False,False,True
